### 보팅 방식 모델 구현 <hr>
- 데이터 : sklearn.datasets의 breast_cancer
- 유형 : 지도학습 + 분류
- 방법 : Voting 방식으로 진행 => LogisticRegression, DecisionTreeClassifier, SVC
- 학습 데이터 셋 : 동일한 데이터셋으로 3개의 모델로 학습 진행

In [109]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(as_frame = True, return_X_y=True)

In [110]:
X.shape, y.shape

((569, 30), (569,))

In [111]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 5, test_size = 0.2)

[2] 학습 진행   <hr>

[2-1] 앙상블 보팅 학습에 사용할 모델 인스턴스 생성

In [112]:
# Logistic regression 인스턴스
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(solver = 'liblinear')     #저 파라미터를 설정하면 max iterater 설정안해도 됨둥
lr_model.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [113]:
# Decision Tree 기반 인스턴스
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [114]:
# SVC 인스턴스
from sklearn.svm import SVC
svc_model = SVC(probability = True)   #SVC는 probalility 파라미터를 설정해줘야 pred_proba 적용 가능하긔 ㅎ.< 
svc_model.fit(X_train, y_train)

SVC(probability=True)

[2-2] Ensemble 알고리즘 기반 분류 

In [115]:
from sklearn.ensemble import VotingClassifier

In [116]:
# 동일 데이터셋으로 병렬학습 진행할 모델 리스트 선정 및 결과 선정 방법 설정
vt_models = VotingClassifier(estimators = [('lr_model', lr_model), ('dt_model', dt_model), ('svc_model', svc_model)],
                            verbose = True)

In [117]:
# 동일 데이터셋을 전달해서 3개의 모델 동시에 학습 진행
vt_models.fit(X_train, y_train)

[Voting] ................. (1 of 3) Processing lr_model, total=   0.0s
[Voting] ................. (2 of 3) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 3) Processing svc_model, total=   0.0s


VotingClassifier(estimators=[('lr_model',
                              LogisticRegression(solver='liblinear')),
                             ('dt_model', DecisionTreeClassifier()),
                             ('svc_model', SVC(probability=True))],
                 verbose=True)

In [118]:
# 예측하기
import pandas as pd
new_data = pd.DataFrame([X_test.iloc[0]], columns = X_test.columns)

X_test.iloc[0].to_frame().T
#vt_models.predict(X_test.iloc[0])  #지금은 hard라서 다수결에 의해서 1개만 나옴

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
525,8.571,13.1,54.53,221.3,0.1036,0.07632,0.02565,0.0151,0.1678,0.07126,...,9.473,18.45,63.3,275.6,0.1641,0.2235,0.1754,0.08512,0.2983,0.1049


In [119]:
# 예측하기
vt_models.predict(new_data)
vt_models.predict_proba(new_data)

AttributeError: predict_proba is not available when voting='hard'

In [ ]:
# 보팅 인스턴스 내의 학습기들
vt_models.estimators_[0].predict(new_data)

In [ ]:
# 보팅 인스턴스 내의 학습기들 => 접근방법 (1)
vt_models.estimators_[0]

In [ ]:
# 보팅 인스턴스 내의 학습기들 => 접근방법 (2)
vt_models.named_estimators_['lr_model'].predict(new_data)
vt_models.named_estimators_.get('lr_model').predict(new_data)

In [ ]:
print(vt_models.named_estimators_['lr_model'].predict(new_data))
print(vt_models.named_estimators_['dt_model'].predict(new_data))
print(vt_models.named_estimators_['svc_model'].predict(new_data))

In [ ]:
for k, v in vt_models.named_estimators_.items():
    print(f"[{k}] => {v.predict(new_data)}")